In [1]:
import py_stringmatching.tokenizer as tk
import string
import nltk
import pickle
from nltk.stem.porter import PorterStemmer
import sys

#-----------------------------------------------------------------------------#
import warnings
warnings.filterwarnings('ignore')
#-----------------------------------------------------------------------------#

import string

import pandas as pd
import pandas.io.sql as psql
import psycopg2 as pg
from datetime import datetime

# First time do: pip install pymongo
from pymongo import MongoClient
from sqlalchemy import create_engine

# import pymongo
# from pymongo import MongoClient


from nltk.corpus import stopwords
nltk.download('stopwords')


nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
import Final_Tweet_Score_Processing

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
## if running a local copy run this command
# mongod
# client = MongoClient('mongodb://localhost:27017/')

In [3]:
# monCollections = db.collection_names()
# db.collection.count(monCollections)

In [2]:
##if connecting to the cluster use the following [sparingly please]
client = MongoClient('3.16.171.5', 27017)
db = client.tweet

monCollections = db.collection_names()

# Connect to database
conn = pg.connect(
    database="dse203", 
    user="postgres", 
    password="newPass"
)

In [3]:
#this function converts mongodb collection names into mapped Postgres variable names
def re_convert(arg):
    yr = '2018'
    ds = '-'
    stem = '-tweet-archiver-bolt_3'
    switcher = {
        "Jan":"01",
        "Feb":"02",
        "Mar":"03",   
    }
    return (yr + switcher.get(arg[:3])+ arg[3:5]+ds+arg[-2:] +stem)

#this function converts Postgres variable  names into mapped mongodb collection names
def date_convert(argument):
    switcher = {
        "1": "Jan",
        "2": "Feb",
        "3": "Mar",
        }
    dayz = argument[6:8]
    return (switcher.get(argument[5]) +  argument[6:8]+"_"+argument[9:11])

#splitting postgres variable name into a partition by (month,day,topic#) 
def query_postgres(arg):
    param = arg.split("_")
    time_partition = param[0]
    topic_num = param[1]
    return time_partition,topic_num

#Query postgres with selected variable
def postgres_variables(topic_num,time_partition):
    hashtags_query = """
    SELECT hashtag FROM hashtag_topic WHERE topic='""" + topic_num + """' AND date='""" + time_partition + """'"""
    topic_hashtags = pd.read_sql(hashtags_query, conn)
    topic_hashtags = list(set(topic_hashtags['hashtag'].values))    
    return topic_hashtags

#querying mongo by looping through a collection and returning all hashtag's texts based upon the postgres query (already segmented by month/day/topic#)
def _querymongo(x,collect):
    topic_hashtags = x
    collect = 'Feb02'
    y = db[collect].aggregate([{'$match': {'entities.hashtags.text': {'$exists': 'true','$in': [topic_hashtags]}},}, {'$project': {'_id': 0, topic_hashtags: "$text"}}])
    return ((list(map(lambda y:y[topic_hashtags],y))))

In [5]:
#iterating through all hashtags in a collection and returning the text associated 
def _run_(day):
    partition = day.split('_')[0]
    hash_variables = postgres_variables(query_postgres(day)[1],query_postgres(day)[0])
    text_hash = []
    for i in hash_variables:
        partitioned_array = (_querymongo(i,re_convert(partition)))
        text_hash.append((query_postgres(day),partitioned_array))   
#         return(text_hash[0][0][0],text_hash[0][0][1],text_hash[0][1])
    for x in text_hash:
        print(x[1],str(x[0][1]),str(x[0][0]))
#         Final_Tweet_Score_Processing.run([x[1]],str(x[0][1]),str(x[0][0]))
        Final_Tweet_Score_Processing.run(list(set(x[1])),str(x[0][1]),str(x[0][0]))

In [39]:
with open('days.txt') as f:
    days = f.read().splitlines()

In [13]:
datasets = []

topic_num = 23
#23 for demo purpose
for day in days:
    for i in range(topic_num):
        if i < 10:
            i = '0' + str(i)
        else:
            i = str(i)
        datasets.append(day+ '_' + i)

In [6]:
for day in days:
    _run_(day)

['Vote @chuckschumer out #VoteSchumerOut https://t.co/vliFFemxKz'] 00 Feb02
['Let’s Elect @MikeTyson to #Congress - at least for a few Rounds. #TruthToPower #GOPComplicit 🥊 https://t.co/3hQwr2Jd2Z', '#VoteEveryTime#stablegenius\n#Karmageddon #GOPTaxFraud\n#GOPTreason  #DenyingDonald\n#GOPGreed\n#TruthToPower https://t.co/sPyvtrGlk6', 'RT @KapeicaResists: #VoteEveryTime#stablegenius\n#Karmageddon #GOPTaxFraud\n#GOPTreason  #DenyingDonald\n#GOPGreed\n#TruthToPower https://t.co/…', 'RT @KapeicaResists: #VoteEveryTime#stablegenius\n#Karmageddon #GOPTaxFraud\n#GOPTreason  #DenyingDonald\n#GOPGreed\n#TruthToPower https://t.co/…'] 00 Feb02
['The election proved that liberals #identitypolitics over our country. You support voter fraud and election fraud (D… https://t.co/jjzDwARozS'] 00 Feb02
['@Sick_of_Bias @realDonaldTrump This depends upon what #Voting #Count #Model will be used at #New #Elections \n\nAs a… https://t.co/EoWd2YcKTi'] 00 Feb02
['Trump LIED about number of viewers for State of 

['Greedy Democratic Socialist Bernie Sanders pays interns #FightFor15 ! \n\nTHIS is a leader #Bernie2020 so damn ready https://t.co/EzwfyTEBEg', 'RT @kimdallas07: Greedy Democratic Socialist Bernie Sanders pays interns #FightFor15 ! \n\nTHIS is a leader #Bernie2020 so damn ready https:/…', '@CBSNews Honest mistake.\n#MAGA #tcot #MolonLabe #Trump #QAnon #Qanon8chan #Bernie2020 #cnn #msnbc… https://t.co/IDZ53dzNKD', 'Drum roll, please... getting a little warmer - can you feel it? #FeelTheBern #Bernie2020 https://t.co/ZyXFfUJKVW', 'In 2020 the DNC better not rig the primaries like they did in favor of Hillary Clinton. #Bernie2020', 'RT @JudyJones1990: hell yeah #Bernie2020 https://t.co/Y1e8z2aZZZ', "RT @zacknora: It's gonna happen people... We're about to get the president this country deserves #Bernie2020 https://t.co/SapeUVNRyV", 'RT @RoseAnnDeMoro: Bernie Sanders talks universal Medicare, and 1.1 million people click to watch him\n#MedicareForAll #Bernie2020 #Wednesda…', 'RT @AlaynasMo

['.@anjanaomkashyap @aajtak #HallaBol Nothing wrong if DG Home Guard being #Hindu swears by construction of… https://t.co/tkRT4qUupm'] 00 Feb02
['Trump the Greatest EVER American soo good it hurts #conmerchant #supreme', '#washingtondc #dc #supreme #court #lawofland @ Supreme Court of the United States https://t.co/CYpnhlnEfI'] 00 Feb02
['#MillennialNegroSpirituals #FlyPhillyFly https://t.co/h3SqdC7RJ7', '@ShafPatel @realDonaldTrump You are a fraud #Subhanallah #MillennialNegroSpirituals', 'RT @thebestcloser: #Disobedience #MillennialNegroSpirituals https://t.co/A4Ox12oW12'] 00 Feb02
['#factCheck U were conned Trump Supporters ‼️‼️#DonTheCon #DonTheCon #DonTheCon #DonTheCon #DonTheCon #DonTheCon… https://t.co/xNKjtZPowr'] 00 Feb02
["#California #californiarepublicans @gopcaucus remind voters of @RepSwalwell when you see your 401k's and paychecks.… https://t.co/Zr4ijnh8jz"] 00 Feb02
['RT @Unpersuaded112: This is NOT #MAGA at all! None of us voted for this except oil and gas executives. 

['@Sick_of_Bias @realDonaldTrump This depends upon what #Voting #Count #Model will be used at #New #Elections \n\nAs a… https://t.co/EoWd2YcKTi'] 00 Feb02
["RT @redhead4645: We have no recollection ! RUSSIA, RUSSIA, RUSSIA! It is Trump's fault. @POTUS right again! #WireTap #Flynn #DrainingTheSwa…", "RT @redhead4645: Let's get the ball rolling.  We don't have to ask for their ok!!! They didn't ask if it was ok ! #WireTap https://t.co/wAb…", "RT @redhead4645: Let's get the ball rolling.  We don't have to ask for their ok!!! They didn't ask if it was ok ! #WireTap https://t.co/wAb…", "RT @redhead4645: Let's get the ball rolling.  We don't have to ask for their ok!!! They didn't ask if it was ok ! #WireTap https://t.co/wAb…", 'RT @WenMaMa2: .🚨Released FISA memo showed #WeThePeople #ObamaGate #WireTap YUGE SCANDAL👉#LockThemAllUp 👉 #DeepStateCollusion #DeepstateCorr…', "RT @redhead4645: We have no recollection ! RUSSIA, RUSSIA, RUSSIA! It is Trump's fault. @POTUS right again! #WireTap #Flynn

['FREE #CEWebcast on #NAFTA. Are the talks doomed to fail or is there still a path ahead? What happens if Nafta colla… https://t.co/3JQhcezVo1'] 00 Feb02
['#ParentShaming people #ShutUp they call #Trump bad\n#Kiss Between #TomBrady, Son Raises Questions About Affection ??… https://t.co/nKjVDyjfSv'] 00 Feb02
['@ShafPatel @realDonaldTrump You are a fraud #Subhanallah #MillennialNegroSpirituals'] 00 Feb02
['#FightOn #USC https://t.co/R1FnA9eUNM State’s election boss beats Trump on voting panel'] 00 Feb02
['The evidence was given today and they still can’t believe it....🤣🤣🤣🤣🤣\n\n#Libtardation https://t.co/x4an2ZwjeR'] 00 Feb02
['A YEAR AGO FORBES+ @REPMAXINEWATERS + @CNN KEPT LIES #BOYCOTTCNN The Trump Dossier Is Fake -- And Here Are The Reas… https://t.co/KktZsz5VLt'] 00 Feb02
['@realDonaldTrump No, not true!!! We see your game. You #MustNotFireRosenstein #MustNotFireRosenstein… https://t.co/gmgzr61vJQ'] 00 Feb02
['Voted #TakePityOnMelania on @TheTylt because nobody deserves what Melania 

...

['Debut for Carlos Sainz Jr. into rallies during Monte 2018. #rally #race #montecarlo #conpetition… https://t.co/4h48JU12kM'] 21 Feb02
['RT @RealJamesWoods: And the other 25% are wiping wallpaper paste off the corners of their mouths... #JoeBlow #PolidentPelosi https://t.co/p…', 'RT @RealJamesWoods: And the other 25% are wiping wallpaper paste off the corners of their mouths... #JoeBlow #PolidentPelosi https://t.co/p…', "RT @BlueSea1964: 🚨 'Drool' On Rep. Joe Kennedy's Mouth Distracts From Dem Response To Trump's State Of The Union! 😂😂😂😂\n\n#JoeBlow\n#SOTU \n#Re…", "RT @BlueSea1964: 🚨 'Drool' On Rep. Joe Kennedy's Mouth Distracts From Dem Response To Trump's State Of The Union! 😂😂😂😂\n\n#JoeBlow\n#SOTU \n#Re…", '#JoeBlow has no legitimate premise to bash @realDonaldTrump. JFK was a American nationalist who love his country an… https://t.co/gLpSUXgr0v', 'RT @RealJamesWoods: And the other 25% are wiping wallpaper paste off the corners of their mouths... #JoeBlow #PolidentPelosi https://t.

['Brenda is horny!\n\n https://t.co/8UXzeilFm1  ←←\n\n#Merchandise #sexmeets #VipParty\n  Bondage in #Kastoria Medemblik Newport News #Trump'] 21 Feb02
['Blue Pill = Fake Peace\nRed Pill = #GreatAwakening &gt; #TruthWillSetUsFree &gt; #TrueFreedom \n#MAGA\n@realDonaldTrump https://t.co/64RjtChyRi'] 21 Feb02
['RT @rickhasen: #ELB: “Koch-aligned group already prepping for next Supreme Court fight” https://t.co/M8TFBSmALF'] 21 Feb02
['@SkyBet  4+ match tons in the snooker match between Ding v Trump #RequestABet'] 21 Feb02
['#StockNews US stocks suffer biggest daily loss of Trump era https://t.co/gv7Ite4FCa'] 21 Feb02
['Diego #Maradona refused entry to #US via @truth_ngo https://t.co/dUB9rHHgOx\n#Trump #DiegoMaradona #soccer #ترمب', 'RT @TurkKick: .@realDonaldTrump @netanyahu During a #soccer #football match in #Konya #Turkey a choraography was performd by so called #Kon…', 'RT @TurkKick: .@realDonaldTrump @netanyahu During a #soccer #football match in #Konya #Turkey a choraography was per

['@AMErikaNGIRLBOT @ABC @realDonaldTrump @StockMonsterVIP #Wow! #CAA? #Crazy!!'] 22 Feb02
['RT @cechv2: #pakistan #india #mumbai #china #Shanghai #Iran #indonesia #Philippines #brazil #nigeria #Kenya #Egypt #Tunisia #mobile #TMobil…', 'RT @LaughOutNOW: "RU HBO WORTHY? IS 2018 UR BREAKOUT YEAR? https://t.co/qpbWPEgLRR #bulldogs #bollywood #mumbai #bangalore #asia #perth #au…'] 22 Feb02
['Chuck Schumer flip-flops, undeniable.\n\nChuck Schumer Literally Flip-Flopping on EVERYTHING - #TFNOriginal… https://t.co/m3eagRfCnw'] 22 Feb02
['#wow https://t.co/EEUmYyaqQC'] 22 Feb02
['Will #PunchANazi become #PunchAFeminist? https://t.co/PHIaXEeHGC'] 22 Feb02
['Trolls?  #Republicons https://t.co/lJKVuhektg', '#MSNBC @MSNBC \n#inners @allinwithchris\n@Maddow @MaddowBlog \n#NRA merged funding #tRump and #Republicons with Russia… https://t.co/NjmEXyFM7M'] 22 Feb02
["Wow! Can you feel America getting 'greater' already?? #MAGATs https://t.co/R4c8gXFlIA"] 22 Feb02
['Would You FUCK an Older Woman?\n\n http

['Bannon’s Babes #JacobReesMogg #borisjohnson #BrexitShambles'] 22 Feb02
['RT @cechv2: #pakistan #india #mumbai #china #Shanghai #Iran #indonesia #Philippines #brazil #nigeria #Kenya #Egypt #Tunisia #mobile #TMobil…'] 22 Feb02
['Found myself a #blistshill in the sunshine with a six pence to spare... what would you buy?! #bakery #chipshop… https://t.co/z0tuNRmKQv', 'Found myself a #blistshill in the sunshine with a six pence to spare... what would you buy?! #bakery #chipshop… https://t.co/mCaeJUdppX'] 22 Feb02
['Would You FUCK an Older Woman?\n\n https://t.co/ow6O278w3m  &lt;&lt;&lt;&lt;\n\n#hotpics #sexyfeet #babes\nfind  hard fucking  Berns… https://t.co/4DT73t7RnR'] 22 Feb02
['Why so #SupremeCourt #india uncomfortable/biased toward #genderneutral #laws in #india\n#Fakecases \n@vaastavngo… https://t.co/fsS0o0BKWy', 'RT @FaridulHaqLive: Why so #SupremeCourt #india uncomfortable/biased toward #genderneutral #laws in #india\n#Fakecases \n@vaastavngo \n@mymumb…', 'RT @FaridulHaqLive: Why 

['Okay lefties,explain this?\n#animalspirits\n#jobsjobsjobs\n#MAGA\n@realDonaldTrump \n@SenSchumer \n@TeamPelosi https://t.co/uvXueIKlGb'] 22 Feb02
['Why so #SupremeCourt #india uncomfortable/biased toward #genderneutral #laws in #india\n#Fakecases \n@vaastavngo… https://t.co/fsS0o0BKWy', 'RT @FaridulHaqLive: Why so #SupremeCourt #india uncomfortable/biased toward #genderneutral #laws in #india\n#Fakecases \n@vaastavngo \n@mymumb…', 'RT @FaridulHaqLive: Why so #SupremeCourt #india uncomfortable/biased toward #genderneutral #laws in #india\n#Fakecases \n@vaastavngo \n@mymumb…', 'RT @FaridulHaqLive: Why so #SupremeCourt #india uncomfortable/biased toward #genderneutral #laws in #india\n#Fakecases \n@vaastavngo \n@mymumb…', 'RT @FaridulHaqLive: Why so #SupremeCourt #india uncomfortable/biased toward #genderneutral #laws in #india\n#Fakecases \n@vaastavngo \n@mymumb…', 'Supreme Court rejects petition seeking to make the crime of #rape #genderneutral', 'RT @FaridulHaqLive: Why so #SupremeCo

In [21]:
x = pd.read_sql("""select * from tweet_text_scores limit 100""", conn)
x

,index,text,topic,partition,newsworthiness,uniqueness
0,0,#trump #MAGA #tcoy #voters https://t.co/qTU15S...,00,Feb02,0.400000,0.600000
1,0,😂😂😂\n🇺🇸is getting #woke\nfast\ntheir echo cham...,00,Feb02,0.571429,0.785714
2,0,#FactCheck #SOTUniom #SOTUِ #ImpeachTrump #You...,00,Feb02,0.000000,1.000000
3,1,RT @justabigkid1: Trump's State of the Union r...,00,Feb02,0.500000,0.583333
4,2,@realDonaldTrump #FactCheck Trump before you ...,00,Feb02,0.285714,0.714286
5,3,RT @Alyssa_Milano: #FactCheck \n\nI believe in...,00,Feb02,0.642857,0.571429
6,4,#FactCheck #SOTUniom #SOTUِ #ImpeachTrump #You...,00,Feb02,0.000000,1.000000
7,5,@realDonaldTrump @FoxNews #FactCheck \n\nI bel...,00,Feb02,0.545455,0.636364
8,0,"RT @_TVLiveTweeter: Tyra: “You voted red, now ...",00,Feb02,0.384615,0.769231
9,0,#DiegoMaradona https://t.co/RZ9nxFZ07I,00,Feb02,0.000000,1.000000


# Output for postgres one day

In [40]:
q1 = pd.read_sql("""select text

from tweet_text_scores

where uniqueness > .8

order by newsworthiness desc

limit 10""",conn)

q1

,text
0,They can't the missile test or the carrier. Wh...
1,Trump to host Australian PM on February 23 - h...
2,"Interesting review, I did not realise Colin Po..."
3,Supreme Court rejects petition seeking to make...
4,CBI Moves Supreme Court Against 12-Year-Old Or...
5,"Before it was Iraq, Syria, #Yemen, #Ukraine an..."
6,RT @Paprika_im_Blut: I want all the troops hom...
7,"speaking out: @THELITTLEIDIOT (February 2, 201..."
8,I followed back all the #Resistance \nIf I mis...
9,@CNN #Topic #Immigrants? So un America. Mr. #P...


In [41]:
q2 = pd.read_sql("""select topic, partition, count(*)

from tweet_text_scores

group by topic, partition

order by partition, topic""", conn)

q2

,topic,partition,count
0,00,Feb02,448
1,01,Feb02,97
2,02,Feb02,25
3,03,Feb02,79
4,04,Feb02,54
5,05,Feb02,95
6,06,Feb02,17
7,07,Feb02,694
8,08,Feb02,60
9,09,Feb02,472


In [42]:
q3 = pd.read_sql("""select * from tweet_text_scores limit 100""", conn)

q3

,index,text,topic,partition,newsworthiness,uniqueness
0,0,#trump #MAGA #tcoy #voters https://t.co/qTU15S...,00,Feb02,0.400000,0.600000
1,0,😂😂😂\n🇺🇸is getting #woke\nfast\ntheir echo cham...,00,Feb02,0.571429,0.785714
2,0,#FactCheck #SOTUniom #SOTUِ #ImpeachTrump #You...,00,Feb02,0.000000,1.000000
3,1,RT @justabigkid1: Trump's State of the Union r...,00,Feb02,0.500000,0.583333
4,2,@realDonaldTrump #FactCheck Trump before you ...,00,Feb02,0.285714,0.714286
5,3,RT @Alyssa_Milano: #FactCheck \n\nI believe in...,00,Feb02,0.642857,0.571429
6,4,#FactCheck #SOTUniom #SOTUِ #ImpeachTrump #You...,00,Feb02,0.000000,1.000000
7,5,@realDonaldTrump @FoxNews #FactCheck \n\nI bel...,00,Feb02,0.545455,0.636364
8,0,"RT @_TVLiveTweeter: Tyra: “You voted red, now ...",00,Feb02,0.384615,0.769231
9,0,#DiegoMaradona https://t.co/RZ9nxFZ07I,00,Feb02,0.000000,1.000000
